<a href="https://colab.research.google.com/github/MarianoChic09/MSc-ORT-Deep-Learning/blob/main/DL_Obligatorio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![AIRBNB](https://www.stevenridercpa.au/wp-content/uploads/2022/09/airbnb-tax.jpeg)

# Obligatorio Deep Learning

# Obligatorio de Deep Learning
## Semestre 2 - 2023
-------

## Problema

Se presenta un dataset que contiene información de alojamientos publicados en AirBnB con sus respectivos precios. El tamaño del dataset de train es de 1.5 Gb aproximadamente, y 0.5 Gb el de test. Este cuenta con 84 variables predictoras que se podrán utilizar como consideren adecuado.

El objetivo es asignar el precio correcto a los alojamientos listados.

Además del dataset se les provee esta notebook conteniendo el script de carga de datos y un modelo baseline que corresponde a una arquitectura feed forward.

------

## Consigna

### A) <u>Participación en Competencia Kaggle</u>:
El objetivo de este punto es participar en la competencia de Kaggle y obtener como mínimo un Mean Absolute Error inferior a 70 puntos. [->Link a la competencia<-](https://www.kaggle.com/t/69c648e3aa214d1f812bf2314c8d4ffa).

### B) <u>Utilización de Grid Search (o equivalente)</u>:
Para cumplir con la busqueda de modelos óptimos se debe realizar un grid search lo más abarcativo y metódico posible. Recomendamos enfáticamente [Weights and Biases](https://wandb.ai/site)

### C) <u>Se debe a su vez investigar e implementar las siguientes técnicas</u>:
#### 1. [Batch Normalization](https://machinelearningmastery.com/how-to-accelerate-learning-of-deep-neural-networks-with-batch-normalization/)
#### 2. [Gradient Normalization y/o Gradient Clipping](https://machinelearningmastery.com/how-to-avoid-exploding-gradients-in-neural-networks-with-gradient-clipping/)


Además como en todas las tareas se evaluará la prolijidad de la entrega, el preprocesamiento de datos, visualizaciones y exploración de técnicas alternativas.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install langdetect
!pip install tqdm
!pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 18.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993224 sha256=b2e3631c6a568c7f637a43f4a0d4519fc285dffa4d0ad87db020c95d0f32930b
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 28.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.7/251.7 kB 30.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.7 MB/s eta 0:00:00


# Installing cudf

In [3]:
!nvidia-smi

Sun Nov 26 02:59:33 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    49W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
# This get the RAPIDS-Colab install files and test check your GPU.  Run this and the next cell only.
# Please read the output of this cell.  If your Colab Instance is not RAPIDS compatible, it will warn you and give you remediation steps.
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!python rapidsai-csp-utils/colab/env-check.py

Cloning into 'rapidsai-csp-utils'...
remote: Enumerating objects: 395, done.
remote: Counting objects: 100% (126/126), done.
remote: Compressing objects: 100% (75/75), done.
remote: Total 395 (delta 92), reused 53 (delta 51), pack-reused 269
Receiving objects: 100% (395/395), 108.50 KiB | 18.08 MiB/s, done.
Resolving deltas: 100% (194/194), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 2.0 MB/s eta 0:00:00
***********************************************************************
Woo! Your instance has the right kind of GPU, a NVIDIA A100-SXM4-40GB!
We will now install RAPIDS via pip!  Please stand by, should be quick...
***********************************************************************



In [ ]:
# This will update the Colab environment and restart the kernel.  Don't run the next cell until you see the session crash.
!bash rapidsai-csp-utils/colab/update_gcc.sh
import os
os._exit(00)

Updating your Colab environment.  This will restart your kernel.  Don't Panic!
Found existing installation: cupy-cuda11x 11.0.0
Uninstalling cupy-cuda11x-11.0.0:
  Successfully uninstalled cupy-cuda11x-11.0.0
PPA publishes dbgsym, you may need to include 'main/debug' component
Repository: 'deb https://ppa.launchpadcontent.net/ubuntu-toolchain-r/test/ubuntu/ jammy main'
Description:
Toolchain test builds; see https://wiki.ubuntu.com/ToolChain

More info: https://launchpad.net/~ubuntu-toolchain-r/+archive/ubuntu/test
Adding repository.
Adding deb entry to /etc/apt/sources.list.d/ubuntu-toolchain-r-ubuntu-test-jammy.list
Adding disabled deb-src entry to /etc/apt/sources.list.d/ubuntu-toolchain-r-ubuntu-test-jammy.list
Adding key to /etc/apt/trusted.gpg.d/ubuntu-toolchain-r-ubuntu-test.gpg with fingerprint 60C317803A41BA51845E371A1E9377A2BA9EF27F
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/rep

In [1]:
# This will install CondaColab.  This will restart your kernel one last time.  Run this cell by itself and only run the next cell once you see the session crash.
import condacolab
condacolab.install()

⏬ Downloading https://github.com/conda-forge/miniforge/releases/download/23.1.0-1/Mambaforge-23.1.0-1-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:08
🔁 Restarting kernel...


In [1]:
# you can now run the rest of the cells as normal
import condacolab
condacolab.check()

✨🍰✨ Everything looks OK!


In [2]:
# Installing RAPIDS is now 'python rapidsai-csp-utils/colab/install_rapids.py <release> <packages>'
# The <release> options are 'stable' and 'nightly'.  Leaving it blank or adding any other words will default to stable.
!python rapidsai-csp-utils/colab/install_rapids.py stable
import os
os.environ['NUMBAPRO_NVVM'] = '/usr/local/cuda/nvvm/lib64/libnvvm.so'
os.environ['NUMBAPRO_LIBDEVICE'] = '/usr/local/cuda/nvvm/libdevice/'
os.environ['CONDA_PREFIX'] = '/usr/local'

Found existing installation: cffi 1.15.1
Uninstalling cffi-1.15.1:
  Successfully uninstalled cffi-1.15.1
Found existing installation: cryptography 40.0.1
Uninstalling cryptography-40.0.1:
  Successfully uninstalled cryptography-40.0.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 446.3/446.3 kB 20.3 MB/s eta 0:00:00
Installing RAPIDS Stable 23.10
Starting the RAPIDS install on Colab.  This will take about 15 minutes.
Solving environment: ...working... done


==> WARNING: A newer version of conda exists. <==
  current version: 23.1.0
  latest version: 23.10.0

Please update conda by running

    $ conda update -n base -c conda-forge conda

Or to minimize the number of packages updated during conda update use

     conda install conda=23.10.0



## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - mamba


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca

# Obligatorio Deep Learning
## 1. Setup
### 1.1 Imports

In [3]:
%cd /content/drive/MyDrive/Colab Notebooks/Datasets

/content/drive/MyDrive/Colab Notebooks/Datasets


In [4]:
# !git clone https://github.com/rapidsai/rapidsai-csp-utils.git
# !python rapidsai-csp-utils/colab/pip-install.py



In [6]:
import cudf

# Path to your CSV file
file_path = '/content/drive/MyDrive/Colab Notebooks/Datasets/obligatorio_DL/public_train_data.csv'

# Read the CSV file using dask_cudf
df = cudf.read_csv(file_path)

In [7]:
df.head(2)

,id,Last Scraped,Name,Summary,Space,Description,Experiences Offered,Neighborhood Overview,Notes,Transit,...,Review Scores Location,Review Scores Value,License,Jurisdiction Names,Cancellation Policy,Calculated host listings count,Reviews per Month,Geolocation,Features,Price
0,0,2017-05-12,Grand Loft in the heart of historic Antwerp,Best location for visiting Antwerp!! Beautiful...,Welcome in Antwerp!! The loft is situated on t...,Best location for visiting Antwerp!! Beautiful...,none,<NA>,<NA>,<NA>,...,10.0,9.0,<NA>,<NA>,strict,2.0,2.6,"51.21938762207894, 4.4034442505151885","Host Has Profile Pic,Instant Bookable",159.0
1,1,2017-05-03,"CHARMING, CLEAN & COZY BUNGALOW!",Very centrally located and less than 15 min fr...,"Well lit, private entrance with small patio.",Very centrally located and less than 15 min fr...,none,"Quiet. Pretty tree lined streets, safe area.",Has dining table and high back desk chair.,"Uber, bus line and metro link is less than 5 m...",...,<NA>,<NA>,<NA>,"City of Los Angeles, CA",flexible,1.0,<NA>,"34.1892692286356, -118.41993491931177","Host Has Profile Pic,Is Location Exact",49.0


In [8]:
df.info()

<class 'cudf.core.dataframe.DataFrame'>
RangeIndex: 326287 entries, 0 to 326286
Data columns (total 85 columns):
 #   Column                          Non-Null Count   Dtype
---  ------                          --------------   -----
 0   id                              326287 non-null  int64
 1   Last Scraped                    326286 non-null  object
 2   Name                            326018 non-null  object
 3   Summary                         315651 non-null  object
 4   Space                           228792 non-null  object
 5   Description                     326188 non-null  object
 6   Experiences Offered             326287 non-null  object
 7   Neighborhood Overview           192513 non-null  object
 8   Notes                           130729 non-null  object
 9   Transit                         200649 non-null  object
 10  Access                          177108 non-null  object
 11  Interaction                     169193 non-null  object
 12  House Rules                    

### 3.4 Estadísticas descriptivas

In [9]:
df.describe()

/usr/local/lib/python3.10/site-packages/cudf/core/dataframe.py:5062: FutureWarning: `datetime_is_numeric` is deprecated. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  warnings.warn(
/usr/local/lib/python3.10/site-packages/cudf/core/series.py:3334: FutureWarning: `datetime_is_numeric` is deprecated and will be removed in a future release. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  warnings.warn(


,id,Host ID,Host Response Rate,Host Listings Count,Host Total Listings Count,Latitude,Longitude,Accommodates,Bathrooms,Bedrooms,...,Review Scores Rating,Review Scores Accuracy,Review Scores Cleanliness,Review Scores Checkin,Review Scores Communication,Review Scores Location,Review Scores Value,Calculated host listings count,Reviews per Month,Price
count,326287.000000,3.262870e+05,250845.000000,325971.000000,325970.000000,326287.000000,326287.000000,326244.000000,325300.000000,325873.000000,...,243160.000000,242584.000000,242732.000000,242378.000000,242710.000000,242423.000000,242347.000000,325689.000000,246983.000000,326287.000000
mean,163143.000000,3.236757e+07,93.408264,9.586000,9.586026,38.042816,-15.323924,3.270764,1.239482,1.358072,...,92.880063,9.524713,9.326067,9.691416,9.708253,9.468215,9.321031,6.881531,1.486211,138.229041
std,94191.087979,3.174572e+07,17.536835,57.399711,57.399797,22.910029,70.101677,2.037446,0.574784,0.921763,...,8.569521,0.855361,1.038858,0.731702,0.723143,0.805116,0.906478,42.025986,1.752082,149.790527
min,0.000000,1.900000e+01,0.000000,0.000000,0.000000,-38.224427,-123.218712,1.000000,0.000000,0.000000,...,20.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,1.000000,0.010000,0.000000
25%,81571.500000,6.869780e+06,98.000000,1.000000,1.000000,38.923154,-73.968081,2.000000,1.000000,1.000000,...,90.000000,9.000000,9.000000,10.000000,10.000000,9.000000,9.000000,1.000000,0.320000,55.000000
50%,163143.000000,2.186737e+07,100.000000,1.000000,1.000000,42.304549,0.090277,2.000000,1.000000,1.000000,...,95.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,1.000000,0.890000,90.000000
75%,244714.500000,4.799166e+07,100.000000,3.000000,3.000000,50.863658,12.342749,4.000000,1.000000,2.000000,...,100.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,2.000000,2.040000,150.000000
max,326286.000000,1.350885e+08,100.000000,1114.000000,1114.000000,55.994889,153.637837,18.000000,8.000000,96.000000,...,100.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,752.000000,223.000000,999.000000


In [10]:
df.columns

Index(['id', 'Last Scraped', 'Name', 'Summary', 'Space', 'Description',
       'Experiences Offered', 'Neighborhood Overview', 'Notes', 'Transit',
       'Access', 'Interaction', 'House Rules', 'Thumbnail Url', 'Medium Url',
       'Picture Url', 'XL Picture Url', 'Host ID', 'Host URL', 'Host Name',
       'Host Since', 'Host Location', 'Host About', 'Host Response Time',
       'Host Response Rate', 'Host Acceptance Rate', 'Host Thumbnail Url',
       'Host Picture Url', 'Host Neighbourhood', 'Host Listings Count',
       'Host Total Listings Count', 'Host Verifications', 'Street',
       'Neighbourhood', 'Neighbourhood Cleansed',
       'Neighbourhood Group Cleansed', 'City', 'State', 'Zipcode', 'Market',
       'Smart Location', 'Country Code', 'Country', 'Latitude', 'Longitude',
       'Property Type', 'Room Type', 'Accommodates', 'Bathrooms', 'Bedrooms',
       'Beds', 'Bed Type', 'Amenities', 'Square Feet', 'Security Deposit',
       'Cleaning Fee', 'Guests Included', 'Extra Peop

In [11]:
df.shape

(326287, 85)

In [12]:
import cudf
import cuml
from cuml.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
# from cudf.core.subword_tokenizer import SubwordTokenizer
# import dask_cudf
# import dask
import numpy as np

unique_value_threshold = 20

# Separate features and target
df_train = df.drop('Price', axis=1)
y = df['Price']

# Identify numerical columns
numerical_cols = df.select_dtypes(include=['float64', 'int64']).columns.tolist()

# Initialize lists for categorical and textual columns
categorical_cols = []
text_cols = []

# Iterate over columns and classify them as categorical or textual based on unique values
for col in df.columns:
    if col not in numerical_cols:
        unique_values = df[col].nunique(dropna=False)
        if unique_values < unique_value_threshold:
            categorical_cols.append(col)
        else:
            text_cols.append(col)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(df_train, y, test_size=0.2, random_state=0)

# Define important text columns
important_text_cols = ['Name', 'Summary', 'Description', 'Neighborhood Overview', 'Transit', 'Access', 'Interaction', 'House Rules']
text_cols_important = [column for column in text_cols if column in important_text_cols]
# Now create the cuDF DataFrame
X_text_train = cudf.DataFrame(X_train[text_cols_important])
X_text_test = cudf.DataFrame(X_test[text_cols_important])

In [13]:
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')

# List of top 10 languages (modify as needed)
languages = ['english', 'spanish', 'chinese', 'arabic', 'portuguese', 'indonesian', 'french', 'russian', 'german']

# Create a combined set of stopwords from these languages
combined_stopwords = set()
for lang in languages:
    combined_stopwords.update(stopwords.words(lang))


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [9]:
import re
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import cudf
import cuml
from cuml.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
import tensorflow as tf
from transformers import TFBertModel, BertTokenizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-uncased')
# bert_model = TFBertModel.from_pretrained('bert-base-multilingual-uncased')
seed = 42

from transformers import DistilBertTokenizer, TFDistilBertModel

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-multilingual-cased')
model = TFDistilBertModel.from_pretrained('distilbert-base-multilingual-cased')


batch_size = 1000  # Adjust this based on your GPU memory
max_length = 300
# Function to clean a batch of text
def clean_batch(batch):
    # Remove HTML tags
    batch = batch.str.replace('<.*?>', '', regex=True)

    # Correct encoding issues and remove special characters
    batch = batch.str.replace("&amp;", "&").replace("&lt;", "<").replace("&gt;", ">")
    batch = batch.str.replace(r'[^a-zA-Z0-9.,!?/:;\"\'\s]', '', regex=True)

    # Replace multiple spaces with a single space
    batch = batch.str.replace(r'\s+', ' ', regex=True).str.strip()

    return batch

# Function to process text data in batches
def process_in_batches(df, column_name,batch_size):
    num_rows = len(df)
    num_batches = np.ceil(num_rows / batch_size).astype(int)

    for i in range(num_batches):
        batch_start = i * batch_size
        batch_end = min(batch_start + batch_size, num_rows)
        batch = df[column_name].iloc[batch_start:batch_end]
        cleaned_batch = clean_batch(batch)
        df[column_name].iloc[batch_start:batch_end] = cleaned_batch

    return df

def preprocess_each_column_in_batches(df,columns,batch_size=1000):
  for col in columns:
    df = process_in_batches(df, col,batch_size)
  return df

def combine_text_columns(df, columns, column_name='combined_text'):
  for col in columns:
    df[col] = df[col].astype(str)
  df[column_name] = ''
  for col in columns:
    df[column_name] = df[column_name] + df[col] + ' '

  df[column_name] = df[column_name].str.strip()
  return df


def batch_encode(texts, max_length, batch_size=512):
    input_ids = []
    attention_masks = []

    # Convert cuDF Series to Pandas Series and handle null values
    if isinstance(texts, cudf.Series):
        texts = texts.to_pandas().fillna("")

    # Ensure all data are strings
    texts = texts.astype(str)

    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]
        encoded = tokenizer.batch_encode_plus(
            batch,
            add_special_tokens=True,
            padding='max_length',
            truncation=True,
            max_length=max_length,
            return_attention_mask=True,
            return_tensors='tf'
        )
        input_ids.append(encoded['input_ids'])
        attention_masks.append(encoded['attention_mask'])

    input_ids = tf.concat(input_ids, axis=0)
    attention_masks = tf.concat(attention_masks, axis=0)
    return input_ids, attention_masks

def fit_scaler(y,scaler):
  scaler.fit(y.to_numpy().reshape(-1, 1))
  return scaler

def preprocess_target(y, scaler):
  labels = scaler.transform(y.to_numpy().reshape(-1,1))
  return labels

def create_dataset(input_ids, attention_masks, labels, batch_size=32):
    dataset = tf.data.Dataset.from_tensor_slices((
        {
            'input_ids': input_ids,
            'attention_mask': attention_masks
        },
        labels
    ))
    dataset = dataset.shuffle(len(labels)).batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return dataset

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertModel: ['vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight']
- This IS expected if you are initializing TFDistilBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFDistilBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


In [10]:
%%time
print("---------------------------Preprocesamiento de Texto---------------------------")
print("--------------Train----------------")
print("Empezando preprocesamiento")
X_train_text_clean = preprocess_each_column_in_batches(X_text_train, important_text_cols)
X_train_text_combined = combine_text_columns(X_train_text_clean,important_text_cols)
print("Terminando limpieza y combinacion de texto")
print("Empezando a hacer encoding de columnas de texto")
X_train_input_ids, X_train_attention_masks = batch_encode(X_train_text_combined['combined_text'],max_length,batch_size=512)
print("Terminadas de procesar las columnas de texto")

price_scaler = StandardScaler()
print("Escalar columna de precio")
price_scaler_fitted = fit_scaler(y_train,price_scaler)
print("Generando labels")
train_labels = preprocess_target(y_train, price_scaler_fitted)
print("Creando dataset")
train_dataset = create_dataset(X_train_input_ids,X_train_attention_masks, train_labels, batch_size=32)

print("--------------Train----------------")
print("Empezando preprocesamiento")
X_test_text_clean = preprocess_each_column_in_batches(X_text_test, important_text_cols)
X_test_text_combined = combine_text_columns(X_test_text_clean,important_text_cols)
print("Terminando limpieza y combinacion de texto")
print("Empezando a hacer encoding de columnas de texto")
X_test_input_ids, X_test_attention_masks = batch_encode(X_test_text_combined['combined_text'],max_length,batch_size=512)
print("Terminadas de procesar las columnas de texto")

print("Generando labels")
test_labels = preprocess_target(y_test, price_scaler_fitted)

print("Creando dataset")
test_dataset = create_dataset(X_test_input_ids,X_test_attention_masks, test_labels, batch_size=32)

CPU times: user 15min 57s, sys: 6.2 s, total: 16min 3s
Wall time: 15min 53s


In [12]:
# Save the TensorFlow Datasets
import pickle

file_path = '/content/drive/MyDrive/Colab Notebooks/Datasets/obligatorio_DL/'

tf.data.Dataset.save(train_dataset, file_path + 'train_dataset_text')
tf.data.Dataset.save(test_dataset, file_path + 'test_dataset_text')

# Save the scaler and encoder
with open(file_path + 'price_scaler.pkl', 'wb') as f:
    pickle.dump(price_scaler_fitted, f)

# with open(file_path + 'encoder.pkl', 'wb') as f:
#     pickle.dump(encoder, f)



In [33]:
# Load the TensorFlow Datasets
train_dataset2 = tf.data.Dataset.load(file_path + 'train_dataset_text')
test_dataset2 = tf.data.Dataset.load(file_path + 'test_dataset_text')

# Load the scaler and encoder
with open(file_path + 'price_scaler.pkl', 'rb') as f:
    price_scaler2 = pickle.load(f)

# with open(file_path + 'encoder.pkl', 'rb') as f:
#     encoder = pickle.load(f)


In [29]:
train_datase2

<_LoadDataset element_spec=({'attention_mask': TensorSpec(shape=(None, 300), dtype=tf.int32, name=None), 'input_ids': TensorSpec(shape=(None, 300), dtype=tf.int32, name=None)}, TensorSpec(shape=(None, 1), dtype=tf.float64, name=None))>

### Kaggle test data

array([[207.5733  ],
       [109.61352 ],
       [118.88889 ],
       ...,
       [118.888885],
       [118.888885],
       [118.888885]], dtype=float32)

## Incorporating Numerical and Categorical data

In [37]:
import cupy as cp

def numerical_preprocessing(df, numerical_cols, scaler,train=False):
  if train:
    df_numerical = scaler.fit_transform(df[numerical_cols])
  else:
    df_numerical = scaler.transform(df[numerical_cols])

  df_numerical = df_numerical.fillna(df_numerical.mean())
  df_numerical_pandas = df_numerical.to_pandas()
  df_numerical_tensor = tf.convert_to_tensor(df_numerical_pandas.values, dtype = tf.float32)

  return df_numerical_tensor, scaler

def categorical_preprocessing(df, categorical_cols, encoder,train=False):
  if train:
    df_categorical_sparse = encoder.fit_transform(df[categorical_cols])
  else:
    df_categorical_sparse = encoder.transform(df[categorical_cols])

  df_categorical_cupy = cp.asarray(df_categorical_sparse.toarray())
  df_categorical_numpy = df_categorical_cupy.get()
  df_categorical_tensor = tf.convert_to_tensor(df_categorical_numpy)
  return df_categorical_tensor, encoder

In [54]:
scaler = cuml.preprocessing.StandardScaler()
encoder = cuml.preprocessing.OneHotEncoder(handle_unknown='ignore')

numerical_cols.remove('Price')

X_train_numerical_tf, scaler = numerical_preprocessing(X_train, numerical_cols, scaler,train=True)
X_test_numerical_tf, _ = numerical_preprocessing(X_test, numerical_cols, scaler)

X_train_categorical_tf, encoder = categorical_preprocessing(X_train, categorical_cols, encoder,train=True)
X_test_categorical_tf, _ = categorical_preprocessing(X_test, categorical_cols, encoder)

encoded_column_names = encoder.get_feature_names(categorical_cols)


In [44]:
def create_dataset_numerical(numerical, labels, batch_size=32):
    dataset = tf.data.Dataset.from_tensor_slices((
        {
            'numerical': numerical,
        },
        labels
    ))
    dataset = dataset.shuffle(len(labels)).batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return dataset

def create_dataset_num_cat(numerical, categorical, labels, batch_size=32):
    dataset = tf.data.Dataset.from_tensor_slices((
        {
            'numerical': numerical,
            'categorical': categorical
        },
        labels
    ))
    dataset = dataset.shuffle(len(labels)).batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return dataset

def create_dataset_all_features(numerical, categorical,input_ids, attention_masks, labels, batch_size=32):
    dataset = tf.data.Dataset.from_tensor_slices((
        {
            'numerical': numerical,
            'categorical': categorical,
            'input_ids': input_ids,
            'attention_mask': attention_masks
        },
        labels
    ))
    dataset = dataset.shuffle(len(labels)).batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return dataset

In [49]:
train_numerical_dataset = create_dataset_numerical(X_train_numerical_tf,
                                                   train_labels)
test_numerical_dataset = create_dataset_numerical(X_test_numerical_tf,
                                                  test_labels)

train_num_cat_dataset = create_dataset_num_cat(X_train_numerical_tf,
                                               X_train_categorical_tf,
                                               train_labels)
test_num_cat_dataset = create_dataset_num_cat(X_test_numerical_tf,
                                              X_test_categorical_tf,
                                              test_labels)


train_all_dataset = create_dataset_all_features(X_train_numerical_tf,
                                                X_train_categorical_tf,
                                                X_train_input_ids,
                                                X_train_attention_masks,
                                                train_labels)

test_all_dataset = create_dataset_all_features(X_test_numerical_tf,
                                               X_test_categorical_tf,
                                               X_test_input_ids,
                                               X_test_attention_masks,
                                               test_labels)

In [50]:
import pickle

file_path = '/content/drive/MyDrive/Colab Notebooks/Datasets/obligatorio_DL/'

tf.data.Dataset.save(train_num_dataset, file_path + 'train_num_dataset')
tf.data.Dataset.save(test_num_dataset, file_path + 'test_num_dataset')
tf.data.Dataset.save(train_num_cat_dataset, file_path + 'train_num_cat_dataset')
tf.data.Dataset.save(test_num_cat_dataset, file_path + 'test_num_cat_dataset')
tf.data.Dataset.save(train_all_dataset, file_path + 'train_all_dataset')
tf.data.Dataset.save(test_all_dataset, file_path + 'test_all_dataset')

# Save the scaler and encoder
with open(file_path + 'scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

with open(file_path + 'encoder.pkl', 'wb') as f:
    pickle.dump(encoder, f)

In [55]:
with open(file_path + 'encoder.pkl', 'wb') as f:
    pickle.dump(encoder, f)

In [ ]:

train_dataset2 = tf.data.Dataset.load(file_path + 'train_dataset_text')
test_dataset2 = tf.data.Dataset.load(file_path + 'test_dataset_text')



In [25]:
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint callback
checkpoint_filepath = 'model_weights_epoch_{epoch:02d}.h5'
model_checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    save_best_only=False,
    save_freq='epoch'
)

In [36]:
train_dataset2

<_LoadDataset element_spec=({'attention_mask': TensorSpec(shape=(None, 300), dtype=tf.int32, name=None), 'input_ids': TensorSpec(shape=(None, 300), dtype=tf.int32, name=None)}, TensorSpec(shape=(None, 1), dtype=tf.float64, name=None))>

In [52]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

num_numerical_features = len(numerical_cols)
num_categorical_features = len(encoded_column_names)

input_ids = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int32, name='input_ids')
attention_mask = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int32, name='attention_mask')
# bert_output = bert_model([input_ids, attention_mask]).last_hidden_state[:, 0, :]
bert_output = bert_model([input_ids, attention_mask]).last_hidden_state[:, 0, :]

bert_dense = tf.keras.layers.Dense(64, activation='relu')(bert_output)

# Numerical Input
numerical_input = tf.keras.layers.Input(shape=(num_numerical_features,), name='numerical')
numerical_dense = tf.keras.layers.Dense(32, activation='relu', kernel_regularizer=l2(0.01))(numerical_input)

# Categorical Input
categorical_input = tf.keras.layers.Input(shape=(num_categorical_features,), name='categorical')
categorical_dense = tf.keras.layers.Dense(32, activation='relu', kernel_regularizer=l2(0.01))(categorical_input)


concatenated = tf.keras.layers.concatenate([bert_dense, numerical_dense, categorical_dense])
# Concatenate all inputs
dropout_layer = Dropout(0.5)(concatenated)

# Additional layers as needed
output_layer = tf.keras.layers.Dense(1)(dropout_layer)

# Build model
model = tf.keras.Model(inputs=[input_ids, attention_mask, numerical_input, categorical_input], outputs=output_layer)


In [53]:
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

model.load_weights('/content/drive/MyDrive/Colab Notebooks/Datasets/model_weights_epoch_06.h5')

In [55]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.optimizers import Adam, SGD

# Define the sweep configuration
sweep_config = {
    'method': 'bayes',  # Bayesian optimization
    'metric': {
        'name': 'val_loss',
        'goal': 'minimize'
    },
    'parameters': {
        'learning_rate': {
            'min': 1e-5,
            'max': 1e-3
        },
        'batch_size': {
            'values': [32, 64, 128]
        },
        # Add other hyperparameters here
    }
}

def train():
    with wandb.init() as run:
        config = run.config

        # Callbacks
        early_stopping = EarlyStopping(monitor='val_loss', patience=3)
        reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=1e-6)
        model_checkpoint_callback = ModelCheckpoint(
            filepath='model_weights_epoch_{epoch:02d}.h5',
            save_weights_only=True,
            save_best_only=False,
            save_freq='epoch'
        )

        # Build and compile your model here using config parameters
        # For example, setting the optimizer
        optimizer = Adam(learning_rate=config.learning_rate)
        # model.compile(optimizer=optimizer, ...)

        # Fit the model
        history = model.fit(train_dataset_all,
                            validation_data=test_all_dataset,
                            epochs=10,  # You can also use config.epochs if defined in sweep_config
                            batch_size=int(config.batch_size),
                            callbacks=[early_stopping, reduce_lr, model_checkpoint_callback]
        )
# Initialize the sweep
sweep_id = wandb.sweep(sweep_config, project=project)

# Run the sweep agent
wandb.agent(sweep_id, function=train, count=10, project=project)

Create sweep with ID: jukt77lt
Sweep URL: https://wandb.ai/marian-ai/obligatorio_dl/sweeps/jukt77lt


wandb: Agent Starting Run: mqkh4aj2 with config:
wandb: 	batch_size: 128
wandb: 	learning_rate: 0.00022787787721444716


Epoch 1/10
3367/8158 [===========>..................] - ETA: 18:44 - loss: 0.4906 - mae: 0.4021

wandb: Ctrl + C detected. Stopping sweep.


In [ ]:
model.save_weights(f'all_features_bert_{epochs}epochs.h5')


## Loading datasets and scalers/encoders

In [14]:
import tensorflow as tf
import pickle
# from tensorflow.keras.layers import Input, Dense, Dropout
# from tensorflow.keras.regularizers import l2
# from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

from transformers import TFDistilBertModel, DistilBertTokenizer
file_path = '/content/drive/MyDrive/Colab Notebooks/Datasets/obligatorio_DL/'

train_num_dataset= tf.data.Dataset.load(file_path + 'train_num_dataset')
test_num_dataset= tf.data.Dataset.load(file_path + 'test_num_dataset')

train_num_cat_dataset= tf.data.Dataset.load(file_path + 'train_num_cat_dataset')
test_num_cat_dataset= tf.data.Dataset.load(file_path + 'test_num_cat_dataset')

train_all_dataset= tf.data.Dataset.load(file_path + 'train_all_dataset')
test_all_dataset= tf.data.Dataset.load(file_path + 'test_all_dataset')

# Load the scaler and encoder
with open(file_path + 'price_scaler.pkl', 'rb') as f:
    price_scaler = pickle.load(f)

with open(file_path + 'encoder.pkl', 'rb') as f:
    encoder = pickle.load(f)

with open(file_path + 'scaler.pkl', 'rb') as f:
    scaler = pickle.load(f)

In [15]:
# tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-multilingual-cased')
bert_model = TFDistilBertModel.from_pretrained('distilbert-base-multilingual-cased')

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertModel: ['vocab_transform.bias', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing TFDistilBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFDistilBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


In [3]:
# Import the W&B Python Library and log into W&B
import wandb

wandb.login()

#Creamos un proyecto en WandB a través de su interfaz
project = "obligatorio_dl"
entity = "marian-ai"

wandb: Currently logged in as: mariano-chicatun (marian-ai). Use `wandb login --relogin` to force relogin


In [16]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau,ModelCheckpoint


def build_model(feature_combination, num_numerical_features, num_categorical_features, bert_model, max_length,dropout_rate = 0.5,l2_regularization=0.001):
    if feature_combination in ['numerical', 'numerical_categorical', 'all_features']:
        # Numerical Input
        numerical_input = Input(shape=(num_numerical_features,), name='numerical')
        numerical_dense = Dense(32, activation='relu', kernel_regularizer=l2(l2_regularization))(numerical_input)

    if feature_combination in ['numerical_categorical', 'all_features']:
        # Categorical Input
        categorical_input = Input(shape=(num_categorical_features,), name='categorical')
        categorical_dense = Dense(32, activation='relu', kernel_regularizer=l2(l2_regularization))(categorical_input)

    if feature_combination == 'all_features':
        # BERT Input
        input_ids = Input(shape=(max_length,), dtype='int32', name='input_ids')
        attention_mask = Input(shape=(max_length,), dtype='int32', name='attention_mask')
        bert_output = bert_model([input_ids, attention_mask]).last_hidden_state[:, 0, :]
        bert_dense = Dense(64, activation='relu')(bert_output)

    # Concatenate all inputs based on the selected features
    if feature_combination == 'numerical':
        concatenated = numerical_dense
    elif feature_combination == 'numerical_categorical':
        concatenated = tf.keras.layers.concatenate([numerical_dense, categorical_dense])
    elif feature_combination == 'all_features':
        concatenated = tf.keras.layers.concatenate([bert_dense, numerical_dense, categorical_dense])

    # Final layers
    dropout_layer = Dropout(0.5)(concatenated)
    output_layer = Dense(1)(dropout_layer)

    # Build model
    inputs = []
    if feature_combination == 'numerical':
        inputs = [numerical_input]
    elif feature_combination == 'numerical_categorical':
        inputs = [numerical_input, categorical_input]
    elif feature_combination == 'all_features':
        inputs = [input_ids, attention_mask, numerical_input, categorical_input]

    model = tf.keras.Model(inputs=inputs, outputs=output_layer)
    return model


In [17]:
sweep_config = {
    'method': 'bayes',  # or any other method you prefer
    'metric': {
        'name': 'val_loss',
        'goal': 'minimize'
    },
    'parameters': {
        'feature_combination': {
            'values': ['numerical', 'numerical_categorical', 'all_features']
        },
        'batch_size': {
            'values': [32, 64]  # example batch sizes
        },
        'dropout_rate': {
            'values': [0.5, 0.3]
        },
        'epochs': {
            'values': [10, 20]  # example epochs
        },
        'l2_regularization': {
                              'values': [0.001, 0.0001]
            }
    }
}

import pickle
import tensorflow as tf

file_path = '/content/drive/MyDrive/Colab Notebooks/Datasets/obligatorio_DL/'

def load_dataset(dataset_name):
    return tf.data.Dataset.load(file_path + dataset_name)

encoded_column_names = encoder.get_feature_names(categorical_cols)


def train():
    with wandb.init() as run:
        config = run.config

        # Load the appropriate datasets
        if config.feature_combination == 'numerical':
            train_dataset = load_dataset('train_num_dataset')
            test_dataset = load_dataset('test_num_dataset')
        elif config.feature_combination == 'numerical_categorical':
            train_dataset = load_dataset('train_num_cat_dataset')
            test_dataset = load_dataset('test_num_cat_dataset')
        elif config.feature_combination == 'all_features':
            train_dataset = load_dataset('train_all_dataset')
            test_dataset = load_dataset('test_all_dataset')

        model = build_model(config.feature_combination, len(numerical_cols), len(encoded_column_names), bert_model, max_length,dropout_rate=config.dropout_rate)

        # Compile the model
        model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

        # Define Callbacks
        checkpoint_filename = f'model_weights_{config.feature_combination}_epoch_{{epoch:02d}}.h5'
        model_checkpoint_callback = ModelCheckpoint(
            filepath=checkpoint_filename,
            save_weights_only=True,
            monitor='val_loss',
            mode='min',
            save_best_only=False,
            save_freq='epoch'
        )
        wandb_callback = wandb.keras.WandbCallback()

        early_stopping = EarlyStopping(monitor='val_loss', patience=3)
        reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=0.001)

        # Fit the model
        model.fit(
            train_dataset,
            validation_data=test_dataset,
            epochs=config.epochs,
            batch_size=config.batch_size,
            callbacks=[early_stopping, reduce_lr, model_checkpoint_callback,wandb_callback]
        )

In [18]:
# Initialize the sweep
max_length = 300
text_cols = [col for col in text_cols if col in important_text_cols]
numerical_cols.remove('Price')

sweep_id = wandb.sweep(sweep_config, project=project)

# Run the sweep agent
wandb.agent(sweep_id, function=train, count=10, project=project)

Create sweep with ID: a47ag95a
Sweep URL: https://wandb.ai/marian-ai/obligatorio_dl/sweeps/a47ag95a


wandb: Agent Starting Run: nr7vnic2 with config:
wandb: 	batch_size: 64
wandb: 	dropout_rate: 0.5
wandb: 	epochs: 10
wandb: 	feature_combination: all_features
wandb: 	l2_regularization: 0.001


wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


Epoch 1/10
8158/8158 [==============================] - ETA: 0s - loss: 0.5970 - mae: 0.4458

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


...

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


INFO:tensorflow:Assets written to: /content/wandb/run-20231126_191429-nrvmh36k/files/model-best/assets


INFO:tensorflow:Assets written to: /content/wandb/run-20231126_191429-nrvmh36k/files/model-best/assets
wandb: Adding directory to artifact (/content/wandb/run-20231126_191429-nrvmh36k/files/model-best)... Done. 0.0s


8158/8158 [==============================] - 25s 3ms/step - loss: 0.4492 - mae: 0.3942 - val_loss: 0.3770 - val_mae: 0.3652 - lr: 0.0010
Epoch 8/10
8158/8158 [==============================] - 25s 3ms/step - loss: 0.4421 - mae: 0.3911 - val_loss: 0.3850 - val_mae: 0.3592 - lr: 0.0010
Epoch 9/10
8158/8158 [==============================] - 25s 3ms/step - loss: 0.4454 - mae: 0.3906 - val_loss: 0.3785 - val_mae: 0.3619 - lr: 0.0010
Epoch 10/10
8158/8158 [==============================] - 25s 3ms/step - loss: 0.4413 - mae: 0.3884 - val_loss: 0.3771 - val_mae: 0.3517 - lr: 0.0010


epoch,▁▂▃▃▄▅▆▆▇█
loss,█▄▃▂▂▁▁▁▁▁
lr,▁▁▁▁▁▁▁▁▁▁
mae,█▄▃▂▂▂▁▁▁▁
val_loss,█▆▅▃▂▄▁▁▁▁
val_mae,█▆▅▄▃▄▃▂▂▁
best_epoch,6
best_val_loss,0.37699
epoch,9
loss,0.44133
lr,0.001


In [ ]:
model.save_weights(f'num_cat_and_trained_bert_{epochs}epochs.h5')


In [ ]:
import os

# Your code here...

# Command to crash the Colab runtime
os._exit(0)

Error in callback <bound method _WandbInit._resume_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7e91c47c7850>> (for pre_run_cell):


### Test with kaggle (Num, Categorical and Text)

In [34]:
# model.load_weights('/content/drive/MyDrive/Colab Notebooks/Datasets/num_cat_and_trained_bert_10epochs.h5')
model.load_weights('/content/drive/MyDrive/Colab Notebooks/Datasets/model_weights_epoch_06.h5')

In [35]:
import cudf
import cuml
import tensorflow as tf
import numpy as np
import pandas as pd
from cuml.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split

# Load the Kaggle test dataset
file_path2 = './obligatorio_DL/private_data_to_predict.csv'
kaggle_test = cudf.read_csv(file_path2)

# Assuming your important text columns are already defined
important_text_cols = ['Name', 'Summary', 'Description', 'Neighborhood Overview', 'Transit', 'Access', 'Interaction', 'House Rules']
kaggle_test_text = kaggle_test[important_text_cols]

# Apply cleaning and encoding to text data
for col in important_text_cols:
    # Assuming process_in_batches is your batch processing function
    kaggle_test_text = process_in_batches(kaggle_test_text,col) #[col].astype(str)

# Combine the cleaned text columns
kaggle_test_text['combined_text'] = ''
for col in important_text_cols:
    kaggle_test_text['combined_text'] = kaggle_test_text['combined_text'] + kaggle_test_text[col] + ' '
    kaggle_test_text['combined_text'] = kaggle_test_text['combined_text'].str.strip()
# Encode the combined text
kaggle_input_ids, kaggle_attention_masks = batch_encode(kaggle_test_text['combined_text'])

# X_kaggle_numerical = kaggle_test[numerical_cols]
# X_kaggle_categorical = kaggle_test[categorical_cols]

# # Apply transformations
# X_kaggle_numerical_transformed = scaler.transform(X_kaggle_numerical)  # Assuming 'scaler' is your StandardScaler
# X_kaggle_categorical_transformed = encoder.transform(X_kaggle_categorical)  # Assuming 'encoder' is your OneHotEncoder

# # Convert to TensorFlow tensors
# X_kaggle_numerical_tf = tf.convert_to_tensor(X_kaggle_numerical_transformed)
# X_kaggle_categorical_tf = tf.convert_to_tensor(X_kaggle_categorical_transformed)





In [36]:
# Assuming kaggle_test is your DataFrame loaded with the test data for Kaggle

# Preprocess Numerical Columns
# Scale the numerical columns using the same scaler as for the training data
kaggle_test_numerical = scaler.transform(kaggle_test[numerical_cols])
kaggle_test_numerical = kaggle_test_numerical.fillna(kaggle_test_numerical.mean())
kaggle_test_numerical_pd = kaggle_test_numerical.to_pandas()
kaggle_test_numerical_tf = tf.convert_to_tensor(kaggle_test_numerical_pd.values, dtype=tf.float32)

# Categorical
kaggle_test_categorical = encoder.transform(kaggle_test[categorical_cols])
kaggle_test_categorical_cupy = cp.asarray(kaggle_test_categorical.toarray())
kaggle_test_categorical_np = kaggle_test_categorical_cupy.get()
kaggle_test_categorical_tf = tf.convert_to_tensor(kaggle_test_categorical_np)

# Now kaggle_test_numerical_tf and kaggle_test_categorical_tf are ready to be used in your model prediction


In [37]:
# Make predictions
predictions = model.predict({
    'input_ids': kaggle_input_ids,
    'attention_mask': kaggle_attention_masks,
    'numerical_data': kaggle_test_numerical_tf,
    'categorical_data': kaggle_test_categorical_tf
})

# Rescale predictions
y_pred = price_scaler.inverse_transform(predictions)

# Prepare output for submission
test_ids = kaggle_test['id']
output = np.hstack((test_ids.to_numpy().reshape(-1, 1), y_pred))
submission_df = pd.DataFrame(output, columns=['id', 'expected'])
submission_df['expected'] = submission_df['expected'].fillna(0)
submission_df.to_csv("output_to_submit.csv", index=False)

5023/5023 [==============================] - 386s 77ms/step


# Pipeline con Wandb

In [ ]:
def preprocess_data(data_type='numerical', df=df, numerical_cols=numerical_cols, categorical_cols=categorical_cols, text_cols=text_cols, important_text_cols=important_text_cols, unique_value_threshold=20, max_length=300):
    # Select the columns based on data_type
    if data_type == 'numerical':
        df_selected = df[numerical_cols]
    elif data_type == 'numerical_categorical':
        df_selected = df[numerical_cols + categorical_cols]
    else:  # 'all'
        df_selected = df[numerical_cols + categorical_cols + text_cols]

    # Apply your existing preprocessing steps here
    # ...

    return X_train, X_test, train_input_ids, test_input_ids, train_attention_masks, test_attention_masks, y_train, y_test


In [ ]:
def build_model(data_type='numerical', num_layers=1, layer_size=32, dropout_rate=0.0, regularization='none', regularization_rate=0.0, optimizer='adam', learning_rate=0.001):
    # Setup inputs based on data_type
    inputs = []
    concatenated_layers = []

    if 'text' in data_type:
        input_ids = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int32, name='input_ids')
        attention_mask = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int32, name='attention_mask')
        bert_output = bert_model([input_ids, attention_mask]).last_hidden_state[:, 0, :]
        bert_dense = tf.keras.layers.Dense(layer_size, activation='relu')(bert_output)
        inputs.extend([input_ids, attention_mask])
        concatenated_layers.append(bert_dense)

    if 'numerical' in data_type:
        numerical_input = tf.keras.layers.Input(shape=(num_numerical_features,), name='numerical_data')
        numerical_dense = tf.keras.layers.Dense(layer_size, activation='relu', kernel_regularizer=l2(regularization_rate))(numerical_input)
        inputs.append(numerical_input)
        concatenated_layers.append(numerical_dense)

    if 'categorical' in data_type:
        categorical_input = tf.keras.layers.Input(shape=(num_categorical_features,), name='categorical_data')
        categorical_dense = tf.keras.layers.Dense(layer_size, activation='relu', kernel_regularizer=l2(regularization_rate))(categorical_input)
        inputs.append(categorical_input)
        concatenated_layers.append(categorical_dense)

    concatenated = tf.keras.layers.concatenate(concatenated_layers)

    # Add additional layers based on num_layers
    for _ in range(num_layers - 1):
        concatenated = tf.keras.layers.Dense(layer_size, activation='relu', kernel_regularizer=l2(regularization_rate))(concatenated)
        concatenated = Dropout(dropout_rate)(concatenated)

    output_layer = tf.keras.layers.Dense(1)(concatenated)

    # Build model
    model = tf.keras.Model(inputs=inputs, outputs=output_layer)

    # Compile model with the given optimizer and learning rate
    if optimizer == 'adam':
        opt = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    elif optimizer == 'sgd':
        opt = tf.keras.optimizers.SGD(learning_rate=learning_rate)
    model.compile(optimizer=opt, loss='mean_squared_error', metrics=['mae'])

    return model


In [ ]:
def train():
    with wandb.init() as run:
        config = run.config

        # Preprocess data
        X_train, X_test, train_input_ids, test_input_ids, train_attention_masks, test_attention_masks, y_train, y_test = preprocess_data(data_type=config.data_type, ...)

        # Build model
        model = build_model(data_type=config.data_type, num_layers=config.num_layers, layer_size=config.layer_size, dropout_rate=config.dropout_rate, regularization=config.regularization, regularization_rate=config.regularization_rate, optimizer=config.optimizer, learning_rate=config.learning_rate)

        # Fit the model
        # ...


# Text only model

In [ ]:
# Define the model
bert_model = TFDistilBertModel.from_pretrained('distilbert-base-multilingual-cased')

def build_model():
    input_ids = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int32, name='input_ids')
    attention_mask = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int32, name='attention_mask')

    bert_output = bert_model([input_ids, attention_mask])
    cls_token_output = bert_output.last_hidden_state[:, 0, :]
    output = tf.keras.layers.Dense(1)(cls_token_output)

    model = tf.keras.Model(inputs=[input_ids, attention_mask], outputs=output)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5, epsilon=1e-8),
                  loss=tf.keras.losses.MeanSquaredError(),
                  metrics=[tf.keras.metrics.MeanAbsoluteError()])
    return model

model = build_model()

epochs=5

history = model.fit(
    train_dataset,
    validation_data=test_dataset,
    epochs=epochs
)
model.save_weights(f'trained_bert_{epochs}.h5')
# Loading model


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertModel: ['vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_layer_norm.bias']
- This IS expected if you are initializing TFDistilBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFDistilBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


Epoch 1/5
 517/8158 [>.............................] - ETA: 29:39 - loss: 0.9974 - mean_absolute_error: 0.6353

KeyboardInterrupt: ignored

In [ ]:
model.save_weights(f'trained_bert_2ymedia.h5')


In [ ]:
model.load_weights('/content/drive/MyDrive/Colab Notebooks/Datasets/trained_bert.h5')


In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
# Assuming 'basic_cleaning' and 'batch_encode' functions are already defined

# Load the test data
file_path2 = './obligatorio_DL/private_data_to_predict.csv'
kaggle_test = cudf.read_csv(file_path2)

# Apply basic cleaning


# Apply cleaning to each important text column
for col in important_text_cols:
    kaggle_test = process_in_batches(kaggle_test, col)




In [ ]:
# kaggle_test = data_for_kaggle.applymap(basic_cleaning)
kaggle_test['combined_text'] = ''

for col in important_text_cols:
    kaggle_test[col] = kaggle_test[col].astype('str')
    kaggle_test['combined_text'] = kaggle_test['combined_text'] + kaggle_test[col] + ' '
    kaggle_test['combined_text'] = kaggle_test['combined_text'].str.strip()

kaggle_input_ids, kaggle_attention_masks = batch_encode(kaggle_test['combined_text'])

# Make predictions


In [ ]:
predictions = model.predict(x=[kaggle_input_ids, kaggle_attention_masks])

# Rescale predictions
y_pred = price_scaler.inverse_transform(predictions)



In [ ]:
# Prepare output
test_ids = kaggle_test['id']
output = np.hstack((test_ids.to_numpy().reshape(-1, 1), y_pred))
df = pd.DataFrame(output, columns=['id', 'expected'])

# Handle any NaNs
df['expected'] = df['expected'].fillna(0)

# Save to CSV
df.to_csv("output_to_submit.csv", index=False)

# Basura

In [ ]:
def create_dataset(input_ids, attention_masks, numerical_data, categorical_data, labels, batch_size=32):
    # Ensure categorical data is a list of tensors, one for each categorical column
    if not isinstance(categorical_data, list):
        categorical_data = [categorical_data]

    # Create a dictionary for all inputs
    input_dict = {
        'input_ids': input_ids,
        'attention_mask': attention_masks,
        'numerical_data': numerical_data
    }

    # Add each categorical feature to the input dictionary
    for i, cat_data in enumerate(categorical_data):
        input_dict[f'cat_input_{i}'] = cat_data

    # Create the TensorFlow dataset
    dataset = tf.data.Dataset.from_tensor_slices((input_dict, labels))
    dataset = dataset.shuffle(len(labels)).batch(batch_size).prefetch(tf.data.AUTOTUNE)

    return dataset
# Assuming categorical_data_train and categorical_data_test are lists of categorical columns
# For each categorical column, convert to a TensorFlow tensor
categorical_data_train = [tf.convert_to_tensor(categorical_data_train[i], dtype=tf.int32) for i in range([num_categorical_cols])]
categorical_data_test = [tf.convert_to_tensor(categorical_data_test[i], dtype=tf.int32) for i in range(num_categorical_cols)]

# Convert numerical data to TensorFlow tensors
numerical_data_train = tf.convert_to_tensor(numerical_data_train, dtype=tf.float32)
numerical_data_test = tf.convert_to_tensor(numerical_data_test, dtype=tf.float32)

# Create the datasets
train_dataset = create_dataset(train_input_ids, train_attention_masks, numerical_data_train, categorical_data_train, train_labels)
test_dataset = create_dataset(test_input_ids, test_attention_masks, numerical_data_test, categorical_data_test, test_labels)


NameError: ignored

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Embedding, Concatenate, Normalization
from tensorflow.keras.models import Model

# Define input layers
input_ids = Input(shape=(max_length,), dtype=tf.int32, name='input_ids')
attention_mask = Input(shape=(max_length,), dtype=tf.int32, name='attention_mask')
numerical_input = Input(shape=(numerical_cols,), name='numerical_data')
categorical_inputs = [Input(shape=(1,), name=f'cat_input_{i}') for i in range(categorical_cols)]

# BERT model for text
bert_output = bert_model([input_ids, attention_mask]).last_hidden_state[:, 0, :]

# Numerical data processing
numerical_layer = Normalization()(numerical_input)
numerical_output = Dense(64, activation='relu')(numerical_layer)

# Categorical data processing
embedding_layers = [Embedding(input_dim=cat_col_vocab_size, output_dim=embedding_dim)(cat_input) for cat_input, embedding_dim in zip(categorical_inputs, embedding_dims)]
flattened_embeddings = [tf.keras.layers.Flatten()(embedding) for embedding in embedding_layers]

# Concatenate all outputs
concatenated = Concatenate()([bert_output, numerical_output] + flattened_embeddings)
dense_layer = Dense(128, activation='relu')(concatenated)
output = Dense(1)(dense_layer)

# Build and compile the model
model = Model(inputs=[input_ids, attention_mask, numerical_input] + categorical_inputs, outputs=output)
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_absolute_error'])

# Train the model
model.fit(train_dataset, validation_data=test_dataset, epochs=5)


TypeError: ignored

In [ ]:
from cudf.core.subword_tokenizer import SubwordTokenizer
from cudf.utils.hash_vocab_utils import hash_vocab

vocab_file = '/content/drive/MyDrive/Colab Notebooks/Datasets/obligatorio_DL/bert-multilingual-cased/multi_cased_L-12_H-768_A-12/vocab.txt'
# Assuming you have the BERT multilingual vocabulary file
# vocab_file = 'bert-base-multilingual-uncased-vocab.txt'
hash_vocab(vocab_file, 'voc_hash.txt')


In [ ]:

# Initialize tokenizer
tokenizer = SubwordTokenizer('voc_hash.txt', do_lower_case=True)

# Tokenize function
def tokenize_text_column(df, col_name, max_length=128):
    tokenized_output = tokenizer(
        df[col_name],
        max_length=max_length,
        padding='max_length',
        truncation=True,
        return_tensors='pt'
    )
    return tokenized_output['input_ids']

# Apply tokenization
for col in text_cols:
    df[col] = tokenize_text_column(df, col)
